In [100]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd 
app_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df=app_df.copy().drop(columns=['EIN','NAME'])
df.head(1) # It has 34299 rows

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1


In [101]:
show_app= df['APPLICATION_TYPE'].value_counts()
show_app= show_app[show_app < 1065]
show_app=show_app.copy()

def update_app(value):
    if value in show_app:
        return 'OTHER'
    else:
        return value
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].apply(update_app)
df= df.drop(df[df['APPLICATION_TYPE'] == 'OTHER'].index)
df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
Name: APPLICATION_TYPE, dtype: int64

In [102]:
show_class= df['CLASSIFICATION'].value_counts().copy()
show_class= show_class[show_class < 1918]
show_class=show_class.copy()

def update_app(value):
    if value in show_class:
        return 'OTHER'
    else:
        return value
df['CLASSIFICATION'] = df['CLASSIFICATION'].apply(update_app)
df= df.drop(df[df['CLASSIFICATION'] == 'OTHER'].index)
df['CLASSIFICATION'].value_counts()

C1000    15093
C2000     6045
C1200     4836
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [103]:
Do_not_touch = ['STATUS','IS_SUCCESSFUL']
Do_touch = [
    'APPLICATION_TYPE',
    'AFFILIATION',
    'CLASSIFICATION',
    'USE_CASE',
    'ORGANIZATION',
    'INCOME_AMT',
    'SPECIAL_CONSIDERATIONS',
    'ASK_AMT'
]

dummies = pd.get_dummies(df[Do_touch])
df= pd.concat([df, dummies], axis=1)
df= df.drop(columns=Do_touch)

In [120]:
X=df.drop(columns='IS_SUCCESSFUL')
y=df['IS_SUCCESSFUL']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [121]:
scaler = StandardScaler()
xs= scaler.fit(X_train)

X_train_scaled = xs.transform(X_train)
X_test_scaled = xs.transform(X_test)

## Compile, Train and Evaluate the Model

In [123]:
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=100, input_dim=36, activation='relu'))#tanh

nn.add(tf.keras.layers.Dense(units=100, activation='relu'))
nn.add(tf.keras.layers.Dense(units=100, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # Binary label prediction

In [124]:
nn.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_76 (Dense)            (None, 100)               3700      
                                                                 
 dense_77 (Dense)            (None, 100)               10100     
                                                                 
 dense_78 (Dense)            (None, 100)               10100     
                                                                 
 dense_79 (Dense)            (None, 100)               10100     
                                                                 
 dense_80 (Dense)            (None, 1)                 101       
                                                                 
Total params: 34101 (133.21 KB)
Trainable params: 34101 (133.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [125]:
nn.compile(
    loss = 'binary_crossentropy', # This loss function is needed for Binary label prediction
    optimizer='adam', # The algorith for gradient descent
    metrics=['accuracy'] # The metrics to improve the model
)

In [126]:

nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
654/654 [==============================] - 4s 4ms/step - loss: 0.5815 - accuracy: 0.7098
Epoch 2/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5694 - accuracy: 0.7190
Epoch 3/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5665 - accuracy: 0.7193
Epoch 4/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5647 - accuracy: 0.7191
Epoch 5/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5639 - accuracy: 0.7205
Epoch 6/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5639 - accuracy: 0.7222
Epoch 7/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5623 - accuracy: 0.7213
Epoch 8/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5621 - accuracy: 0.7220
Epoch 9/100
654/654 [==============================] - 2s 3ms/step - loss: 0.5612 - accuracy: 0.7223
Epoch 10/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5610 - accura

# After multiple tries the best I can find was 72.86
- Even I try with a low data set, droping all the rows named 'OTHER' and tryin with different hidden layers the result of accuracy continue n ot arriving to 75 or above.

In [129]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

218/218 - 0s - loss: 0.6000 - accuracy: 0.7119 - 453ms/epoch - 2ms/step
Loss: 0.5999714732170105, Accuracy: 0.7118887305259705


In [130]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
import h5py
hdf5_filename = 'AlphabetSoupCharity_Optimization.h5'
nn.save(hdf5_filename)
print('Model saved to : ',hdf5_filename)

Model saved to :  AlphabetSoupCharity_Optimization.h5
